In [1]:
import requests
import numpy as np
import cv2

In [2]:
# Fetch Custom Vision endpoint URL and prediction key
ENDPOINT_URL = "https://cvshetty001-prediction.cognitiveservices.azure.com/customvision/v3.0/Prediction/df319196-9dd6-47c5-82eb-dbd41d67d535/detect/iterations/Iteration4/image"  
PREDICTION_KEY = "1a875b042c364ddc910f2ce535ce5f8c"

In [3]:
# Function to get prediction results from Custom Vision
def predict(image_data):
    headers = {
        'Prediction-Key': PREDICTION_KEY,
        'Content-Type': 'application/octet-stream'
    }
    response = requests.post(ENDPOINT_URL, headers=headers,  data=image_data)
    response.raise_for_status()
    results = response.json()['predictions']
    return results

In [4]:
# Load the video file
cap = cv2.VideoCapture('C:/Users/vinit/Test_video1.mp4')

# Set up the output video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('final_output.mp4', fourcc, 30, (int(cap.get(3)), int(cap.get(4))))

In [5]:
# Loop through each frame of the video
while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Convert the frame to a byte array
    _, img_encoded = cv2.imencode('.jpg', frame)
    image_data = img_encoded.tobytes()
    
    # Get prediction results for the current frame
    results = predict(image_data)
    # Loop through the results and draw bounding boxes around detected vehicles
    for result in results:
        if result['probability'] > 0.2:
            tag_name = result['tagName']
            if tag_name == 'Bicycle' or tag_name == 'Car' or tag_name == 'Tram':
                x = int(result['boundingBox']['left'] * cap.get(3))
                y = int(result['boundingBox']['top'] * cap.get(4))
                w = int(result['boundingBox']['width'] * cap.get(3))
                h = int(result['boundingBox']['height'] * cap.get(4))
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
                label = result['tagName']
                confidence = result['probability']
                text = f"{label} ({confidence:.2f})"
                cv2.putText(frame, text, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            
    # Write the annotated frame to the output video
    out.write(frame)
    # Show the annotated frame in a window
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) == ord('q'):
        break

In [6]:
# Release the resources
cap.release()
out.release()
cv2.destroyAllWindows()